In [1]:
!pip install loralib
!pip install sentencepiece

In [2]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AdamW, AutoModelWithLMHead, get_scheduler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
from tqdm.auto import tqdm
import loralib as lora
import mylora
import re
import time

In [3]:
f = open("chinese.txt", "r")
cndata = f.readlines()
f.close()

f = open("english.txt", "r")
endata = f.readlines()
f.close()

print(len(cndata))
print(len(endata))

mydata = [{"cn": cn.strip(), "en": en.strip()} for cn, en in zip(cndata, endata)]
print(mydata[114514])

252777
252777
{'cn': '除了评估之前错误的原因之外，报告指出，任何一个新兴经济体的决策者都应该在推行产业政策之前回答三个基本问题：', 'en': 'Beyond assessing what went wrong previously, the report identifies three fundamental questions that policymakers in any emerging economy should answer before pursuing industrial policy:'}


In [4]:
class Mydataset(Dataset):
    def __init__(self, mydata) -> None:
        super().__init__()
        self.data = mydata
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        return self.data[index]

mydata = Mydataset(mydata)
train_size = int(0.8 * 252777)
val_size = 252777 - train_size
trainset, valset = random_split(mydata, lengths=[train_size, val_size])
print(len(trainset), len(valset))
print(trainset[114514])

202221 50556
{'cn': 'lamp#160;amp#160;amp#160;amp#160;amp#160;amp#160;amp#160; 中国有大约1000家太阳热能公司，营业收入25亿美元，下属60万工人从事制造和安装工作。', 'en': '·China has some 1,000 solar thermal energy firms, generating sales of $2.5 billion and employing 600,000 workers in manufacturing and installation.'}


In [5]:
model_checkpoint = "Helsinki-NLP/opus-mt-zh-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [6]:
cn = trainset[114514]["cn"]
en = trainset[114514]["en"]
print(cn)
print(en)
cn_token = tokenizer(cn)
with tokenizer.as_target_tokenizer():
    en_token = tokenizer(en)

print(cn_token)
print(en_token)
print(tokenizer.convert_ids_to_tokens(cn_token['input_ids']))
print(tokenizer.convert_ids_to_tokens(en_token['input_ids']))

lamp#160;amp#160;amp#160;amp#160;amp#160;amp#160;amp#160; 中国有大约1000家太阳热能公司，营业收入25亿美元，下属60万工人从事制造和安装工作。
·China has some 1,000 solar thermal energy firms, generating sales of $2.5 billion and employing 600,000 workers in manufacturing and installation.
{'input_ids': [3993, 15466, 3631, 11636, 25, 15466, 3631, 11636, 25, 15466, 3631, 11636, 25, 15466, 3631, 11636, 25, 15466, 3631, 11636, 25, 15466, 3631, 11636, 25, 15466, 3631, 11636, 25, 7214, 123, 3351, 29559, 1208, 13812, 4567, 533, 1026, 2, 22722, 1497, 666, 2051, 2, 18430, 1839, 1810, 3510, 1917, 4726, 16, 9587, 193, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [7, 222, 17574, 64, 239, 11616, 20845, 45627, 1504, 10861, 2, 17145, 10813, 4, 51998, 3152, 6, 28230, 52853, 1748, 10, 11838, 6, 18074, 5, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
from transformers import AutoModelForSeq2SeqLM
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, force_download=True, resume_download=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
print(model)

cuda


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [9]:
for name, module in model.named_modules():
    if ('self_attn' in name or 'self_attention' in name) and isinstance(module, nn.Linear):
        print(name)

model.encoder.layers.0.self_attn.k_proj
model.encoder.layers.0.self_attn.v_proj
model.encoder.layers.0.self_attn.q_proj
model.encoder.layers.0.self_attn.out_proj
model.encoder.layers.1.self_attn.k_proj
model.encoder.layers.1.self_attn.v_proj
model.encoder.layers.1.self_attn.q_proj
model.encoder.layers.1.self_attn.out_proj
model.encoder.layers.2.self_attn.k_proj
model.encoder.layers.2.self_attn.v_proj
model.encoder.layers.2.self_attn.q_proj
model.encoder.layers.2.self_attn.out_proj
model.encoder.layers.3.self_attn.k_proj
model.encoder.layers.3.self_attn.v_proj
model.encoder.layers.3.self_attn.q_proj
model.encoder.layers.3.self_attn.out_proj
model.encoder.layers.4.self_attn.k_proj
model.encoder.layers.4.self_attn.v_proj
model.encoder.layers.4.self_attn.q_proj
model.encoder.layers.4.self_attn.out_proj
model.encoder.layers.5.self_attn.k_proj
model.encoder.layers.5.self_attn.v_proj
model.encoder.layers.5.self_attn.q_proj
model.encoder.layers.5.self_attn.out_proj
model.decoder.layers.0.self_

In [10]:
print(model.model.encoder.layers[0].self_attn)

MarianAttention(
  (k_proj): Linear(in_features=512, out_features=512, bias=True)
  (v_proj): Linear(in_features=512, out_features=512, bias=True)
  (q_proj): Linear(in_features=512, out_features=512, bias=True)
  (out_proj): Linear(in_features=512, out_features=512, bias=True)
)


In [8]:
for name, module in model.named_modules():
    if ('self_attn' in name or 'self_attention' in name) and isinstance(module, nn.Linear):
        #print(module.in_features, module.out_features)
        #take model.encoder.layers.0.self_attn.k_proj as example, the out put is 512 512
        parent_name, child_name = name.rsplit('.', 1)
        #print(parent_name, child_name)
        #model.encoder.layers.0.self_attn k_proj
        parent_name = re.sub(r"\.(\d)", r"[\1]", parent_name)
        #model.encoder.layers[0].self_attn
        setattr(eval(f'model.{parent_name}'), child_name, mylora.LoRALinear(module.in_features, module.out_features, r=8))
print(model)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): LoRALinear(in_features=512, out_features=512, bias=True)
            (v_proj): LoRALinear(in_features=512, out_features=512, bias=True)
            (q_proj): LoRALinear(in_features=512, out_features=512, bias=True)
            (out_proj): LoRALinear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((

In [25]:
print(model.model.encoder.layers[0].self_attn.k_proj)
print(model.model.encoder.layers[0].self_attn.k_proj.__class__)
print(model.model.encoder.layers[0].fc1.__class__)
#successfully changed to lora layer

LoRALinear(in_features=512, out_features=512, bias=True)
<class 'mylora.LoRALinear'>
<class 'torch.nn.modules.linear.Linear'>


In [9]:
for name, module in model.named_modules():
    if isinstance(module, mylora.LoRALinear):
        print(name)
    else:
        module.requires_grad_ = False
model = model.to(device)

model.encoder.layers.0.self_attn.k_proj
model.encoder.layers.0.self_attn.v_proj
model.encoder.layers.0.self_attn.q_proj
model.encoder.layers.0.self_attn.out_proj
model.encoder.layers.1.self_attn.k_proj
model.encoder.layers.1.self_attn.v_proj
model.encoder.layers.1.self_attn.q_proj
model.encoder.layers.1.self_attn.out_proj
model.encoder.layers.2.self_attn.k_proj
model.encoder.layers.2.self_attn.v_proj
model.encoder.layers.2.self_attn.q_proj
model.encoder.layers.2.self_attn.out_proj
model.encoder.layers.3.self_attn.k_proj
model.encoder.layers.3.self_attn.v_proj
model.encoder.layers.3.self_attn.q_proj
model.encoder.layers.3.self_attn.out_proj
model.encoder.layers.4.self_attn.k_proj
model.encoder.layers.4.self_attn.v_proj
model.encoder.layers.4.self_attn.q_proj
model.encoder.layers.4.self_attn.out_proj
model.encoder.layers.5.self_attn.k_proj
model.encoder.layers.5.self_attn.v_proj
model.encoder.layers.5.self_attn.q_proj
model.encoder.layers.5.self_attn.out_proj
model.decoder.layers.0.self_

In [10]:
#double check
for name, module in model.named_modules():
    if module.requires_grad_:
        print(name)

model.encoder.layers.0.self_attn.k_proj
model.encoder.layers.0.self_attn.v_proj
model.encoder.layers.0.self_attn.q_proj
model.encoder.layers.0.self_attn.out_proj
model.encoder.layers.1.self_attn.k_proj
model.encoder.layers.1.self_attn.v_proj
model.encoder.layers.1.self_attn.q_proj
model.encoder.layers.1.self_attn.out_proj
model.encoder.layers.2.self_attn.k_proj
model.encoder.layers.2.self_attn.v_proj
model.encoder.layers.2.self_attn.q_proj
model.encoder.layers.2.self_attn.out_proj
model.encoder.layers.3.self_attn.k_proj
model.encoder.layers.3.self_attn.v_proj
model.encoder.layers.3.self_attn.q_proj
model.encoder.layers.3.self_attn.out_proj
model.encoder.layers.4.self_attn.k_proj
model.encoder.layers.4.self_attn.v_proj
model.encoder.layers.4.self_attn.q_proj
model.encoder.layers.4.self_attn.out_proj
model.encoder.layers.5.self_attn.k_proj
model.encoder.layers.5.self_attn.v_proj
model.encoder.layers.5.self_attn.q_proj
model.encoder.layers.5.self_attn.out_proj
model.decoder.layers.0.self_

In [11]:
max_input_length = 128
max_target_length = 128

def mycollate_fn(batch_samples):
    batch_inputs, batch_targets = [], []
    for sample in batch_samples:
        batch_inputs.append(sample['cn'])
        batch_targets.append(sample['en'])
    batch_data = tokenizer(
        batch_inputs,
        padding=True,
        max_length=max_input_length,
        truncation=True,
        return_tensors="pt"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch_targets,
            padding=True,
            max_length=max_target_length,
            truncation=True,
            return_tensors="pt"
        )["input_ids"]
        batch_data['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(labels)
        end_token_index = torch.where(labels == tokenizer.eos_token_id)[1]
        for idx, end_idx in enumerate(end_token_index):
            labels[idx][end_idx+1:] = -100
        batch_data['labels'] = labels
    return batch_data

train_dataloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=mycollate_fn)
valid_dataloader = DataLoader(valset, batch_size=32, shuffle=False, collate_fn=mycollate_fn)

In [12]:
for batch in train_dataloader:
    break
print(batch.keys())
for k, v in batch.items():
    print(k, v.shape)

dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'labels'])
input_ids torch.Size([32, 49])
attention_mask torch.Size([32, 49])
decoder_input_ids torch.Size([32, 61])
labels torch.Size([32, 61])


In [13]:
outputs = model(**batch.to(device))
for k, v in outputs.items():
    print(k, v.shape)

loss torch.Size([])
logits torch.Size([32, 61, 65001])
encoder_last_hidden_state torch.Size([32, 49, 512])


In [14]:
optimizer = AdamW(filter(lambda p: p.requires_grad_, model.parameters()), lr=3e-5)

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))
progress_bar.set_description(f'loss: {0:>7f}')

total_loss = 0.0
losses = []

i = 1
model.train()
for epoch in range(num_epochs):
    t1 = time.time()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        total_loss += loss.item()
        progress_bar.set_description(f'loss: {total_loss/i:>7f}')
        progress_bar.update(1)
        i += 1
    t2 = time.time()
    print("epoch {}, training time {}".format(i, t2-t1))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12640 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "/content/cn-en_translator.pt")

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
preds, labels = [], []
import numpy as np
from sacrebleu.metrics import BLEU
bleu = BLEU()

model.eval()
for batch_data in tqdm(valid_dataloader):
    batch_data = batch_data.to(device)
    with torch.no_grad():
        generated_tokens = model.generate(
            batch_data["input_ids"],
            attention_mask=batch_data["attention_mask"],
            max_length=max_target_length,
        ).cpu().numpy()
    label_tokens = batch_data["labels"].cpu().numpy()

    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)

    preds += [pred.strip() for pred in decoded_preds]
    labels += [[label.strip()] for label in decoded_labels]
    print(preds[0])
    print(labels[0])
    break
bleu_score = bleu.corpus_score(preds, labels).score
print(f"BLEU: {bleu_score:>0.2f}\n")

  0%|          | 0/1580 [00:00<?, ?it/s]

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')
Why should corporate leaders invest in an uncertain world, rather than pay dividends to meet high-demand (but generally risk-averse) investors, or buy back their own shares (and thus improve their rates of return, if only by raising their own salaries)?
['Why would business leaders invest in an uncertain world, rather than paying dividends to demanding (but generally risk-averse) investors, or buying back some of their companies’ own shares (thereby improving the price/earnings ratio and, better yet, increasing their own remuneration)?']
BLEU: 40.01



In [ ]:
def translate_sentence(model, tokenizer, sentence, device, max_target_length=128):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=max_target_length)
    print(inputs)
    # Move tensors to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate translation using the model
    with torch.no_grad():
        generated_tokens = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_target_length,
        ).cpu().numpy()

    # Decode the generated tokens to a string
    decoded_translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    return decoded_translation

# Example usage
model.eval() # Make sure the model is in evaluation mode
sentence = ["你好，世界", "你爱我我爱你，蜜雪冰城甜蜜蜜", "不要文革要改革 不要领袖要选票 不做奴才做公民"] # Your Chinese sentence
translation = translate_sentence(model, tokenizer, sentence, device)
print(translation)


{'input_ids': tensor([[ 5349,     2,   907,     0, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000],
        [  132, 28609, 41412,     2, 16351,  9128,  8677,  5257, 47772, 16351,
             0, 65000, 65000, 65000, 65000, 65000],
        [ 3494,  1926, 27436,   342,  1268,  3494, 10276,   342, 22696,   768,
          1090, 24359,  1880,  1090,  1860,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['Hello, the World.', 'You love me. I love you, honey.', 'No reform of the Cultural Revolution, no vote for the leader, no slavery for the citizen.']
